In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
from ultralytics import YOLO

model = YOLO('yolo11n')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 67.4MB/s]


In [3]:
video_path = '/content/people-detection.mp4'

In [4]:
output_path = 'output_video.avi'

Counting people

In [7]:
import cv2
import numpy as np
from ultralytics import YOLO

def signed_distance(point, line):
  """Calculate the distance between from a point until defined line by two points.
  Allows know what side of the line the point it's.
  point: (x,y)
  line ((x1,y1),(x2,y2))"""
  x,y = point
  (x1,y1),(x2,y2) = line
  num = (y2-y1)*x + (x2-x1)*y + (x2*y1-x1*y2)
  den = np.sqrt((y2-y1)**2 + (x1-x2)**2)
  return num/den

#Define the counting lines
line1 = ((130, 120), (25,300))
line2 = ((650,175), (720,225))

#Conters for each line
count_line1 = 0
count_line2 = 0

#Threshold to associate detections among frames
distance_threshold = 25 # Solving problems associated a detect multiple pople when they are not there

#Array to almacenate the centroids of before frame
prev_centroids = []

model = YOLO('yolo11n.pt')

#Open the input video and configurate videowriter to output video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
  raise ValueError("Unable to open video file")

fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

while True:
  ret, frame = cap.read()
  if not ret:
    break

  #Detect objects
  results = model(frame, conf= 0.7)
  #Acces to fris result of array
  boxes_obj = results[0].boxes

  current_centroids = []
  if boxes_obj is not None:
    #Extract boxes and clasess as numpy arrays
    bboxes = boxes_obj.xyxy.cpu().numpy() #Shape (N,4)
    classes = boxes_obj.cls.cpu().numpy() #Shape (N,)
    #Filtering detections by "person"
    for i in range(len(bboxes)):
      if int(classes[i]) == 0:
        x1, y1, x2, y2 = map (int, bboxes[i])
        centroid = ((x1+x2)//2, (y1+y2)//2)
        current_centroids.append(centroid)
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.circle(frame, centroid, 4, (0,255,0), -1)

  #Drawing lines of counting
  cv2.line(frame, line1[0], line1[1], (255,0,0), 2)
  cv2.line(frame, line2[0], line2[1], (0,255,0), 2)

  #Compare for each current centroid, with the before frame to detect crosses
  for curr in current_centroids:
    best_distance = float('inf')
    best_prev = None
    for prev in prev_centroids:
      d = np.linalg.norm(np.array(curr) - np.array(prev))
      if d < best_distance:
        best_distance = d
        best_prev = prev
      if best_prev is not None:
        #Verify crosses line1
        prev_side1 = signed_distance(best_prev, line1)
        curr_side1 = signed_distance(curr, line1)
        if prev_side1 * curr_side1 < 0:
          count_line1 += 1

        #Verify crosses line2
        prev_side2 = signed_distance(best_prev, line2)
        curr_side2 = signed_distance(curr, line2)
        if prev_side2 * curr_side2 < 0:
          count_line2 += 1

  #Draw the conters in the frame
  cv2.putText(frame, f'Clothes section: {count_line1}', (10,30), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
  cv2.putText(frame, f'Sports section: {count_line2}', (10,70), cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

  #Write the processed frame in the output video
  writer.write(frame)

  #Update centroids
  prev_centroids = current_centroids.copy()

cap.release()
writer.release()
print(f"Processed video and saved on: {output_path}")


0: 384x640 (no detections), 162.6ms
Speed: 7.1ms preprocess, 162.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 168.4ms
Speed: 5.5ms preprocess, 168.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 151.9ms
Speed: 4.7ms preprocess, 151.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 149.6ms
Speed: 5.2ms preprocess, 149.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 152.6ms
Speed: 5.8ms preprocess, 152.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 135.1ms
Speed: 3.8ms preprocess, 135.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 148.8ms
Speed: 5.2ms preprocess, 148.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 166.6ms
Speed: 3.7ms prepr